# Autonomous trackbar
This is the autonomous trackbar method

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pytesseract
import subprocess

# Path to tesseract executable (in case it isn't in your PATH)
try:
    subprocess.call(["tesseract"])
except FileNotFoundError:
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

"""
Automatically modify the values (lh, ls, lv, uh, us, uv) to OCR the text in the image.
"""


def auto_trackbar_ocr(image):
    # Ask user for background color
    background_color = int(input("Enter the background color (1 for Dark and 0 for Light): "))

    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # define range of boundaries color in HSV
    bg_image = np.copy(hsv)  # hsv[:int(image.shape[0] / 5), :int(image.shape[1] / 5)]
    # bg_image = cv2.GaussianBlur(bg_image, (15, 15), 0)
    # bg_image = cv2.medianBlur(bg_image, 31)

    h, s, v = cv2.split(bg_image)
    if background_color == 0:
        lower_value = [np.min(h), np.min(s), np.min(v)]
        upper_value = [180, 255, 255]
    else:
        lower_value = [0, 0, 0]
        upper_value = [np.max(h), np.max(s), np.max(v)]

    lower_value = [np.min(h), np.min(s), np.min(v)]
    upper_value = [np.max(h), np.max(s), np.max(v)]
    print(lower_value, upper_value)
    print(cv2.cvtColor(np.uint8([[[255, 255, 255]]]), cv2.COLOR_BGR2HSV))

    # lower_value = [49, 34, 216] # values per sandisk image 007
    # upper_value = [179, 60, 255] # values per sandisk image 007
    # [H-10, 100,100] and [H+10, 255, 255]

    lower_bound = np.array(lower_value, np.uint8)
    upper_bound = np.array(upper_value, np.uint8)

    # Threshold the HSV to get only the written text
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    # mask = cv2.bitwise_not(mask)
    # mask = cv2.GaussianBlur(mask, (5, 5), 0)
    # mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5)))

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image, image, mask=mask)
    print("Text before filtering: ")
    print("\033[91m {}\033[00m".format(pytesseract.image_to_string(image)))
    # OCR
    res = cv2.GaussianBlur(res, (9, 9), 0)
    text = pytesseract.image_to_string(res, lang='eng')
    print("Text after autonomous trackbar: \033[92m {}\033[00m".format(text))

    hist, bins = np.histogram(hsv.flatten(), 256, (0, 256))

    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()

    plt.plot(cdf_normalized, color='b')
    plt.hist(img.flatten(), 256, (0, 256), color='r')
    plt.xlim([0, 256])
    plt.legend(('cdf', 'histogram'), loc='upper left')
    plt.show()
    plt.subplot(121), plt.imshow(mask), plt.title("Original")
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(res), plt.title("Result")
    plt.xticks([]), plt.yticks([])
    plt.show()


In [ ]:
if __name__ == "__main__":
    plt.rcParams['figure.figsize'] = [15, 10]
    img = cv2.imread('../images/001.png')
    # img = cv2.imread('../images/007.jpg')
    assert img is not None, "file could not be read, check with os.path.exists()"
    auto_trackbar_ocr(img)